In [1]:
import ee
import sys
sys.path.insert(1, "../apGIS/")
import apgis
from apgis import Date
from apgis.apfield import Field

In [2]:
import apgis.geespatial as spatial

In [3]:
apgis.eeInitialize()
Field.showRemotes()

1.karayambedu1_L.geojson
2.karayambedu2_S.geojson
3.karayambedu3_S.geojson
4.karayambedu4_P.geojson
5.lanka_XXXL.geojson
6.mambakkam1_L.geojson
7.mettupalayam1_S.geojson
8.mettupalayam2_S.geojson
9.mettupalayam3_S.geojson
10.mettupalayam4_M.geojson
11.mettupalayam5_P.geojson
12.valparai1_M.geojson
13.valparai2_P.geojson


In [4]:
remotefileName = "mettupalayam4_M"

drange = (Date("2020-08-10"), Date("2020-08-30"))
resource = Field(remotefile=f"{remotefileName}.geojson")

S2 = apgis.gee.genCollection(sensor="L2A", daterange=drange, geometry=resource.eeROI)
print(S2.size().getInfo())

image = S2.first().clip(resource.eeROI.buffer(2000))

4


In [5]:
ndvi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDVI").clip(resource.eeAOI)

In [6]:
ndmi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDMI").clip(spatial.genBufferGeo(resource, 50))

In [7]:
ndviRender = spatial.renderNDVI(ndvi)
ndmiRender = spatial.renderNDMI(ndmi)

In [8]:
ndviLayerRender = spatial.layerCoding(ndviRender)
ndmiLayerRender = spatial.layerCoding(ndmiRender)

In [9]:
ndviLayerCol = spatial.mergeLayerPolygons(ndviLayerRender)
ndmiLayerCol = spatial.mergeLayerPolygons(ndmiLayerRender)

In [10]:
indexes = ndvi.addBands(ndmi)

In [11]:
rawData = spatial.accumulateRawValue(indexes, resource)
scoredData = spatial.scoreBuilder(rawData)

In [12]:
indexes.getInfo()

{'type': 'Image',
 'bands': [{'id': 'NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [42, 65],
   'origin': [9897, 8981],
   'crs': 'EPSG:32644',
   'crs_transform': [10, 0, 300000, 0, -10, 1500000]},
  {'id': 'NDMI',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [87, 86],
   'origin': [9875, 8970],
   'crs': 'EPSG:32644',
   'crs_transform': [10, 0, 300000, 0, -10, 1500000]}],
 'properties': {'system:time_start': 1597295722117,
  'system:footprint': {'type': 'Polygon',
   'coordinates': [[[80.070269, 12.754087999999998],
     [80.069357, 12.750215999999998],
     [80.071535, 12.749462999999999],
     [80.07224300000001, 12.748918],
     [80.073134, 12.749170000000001],
     [80.072329, 12.753251],
     [80.072458, 12.753387000000002],
     [80.072297, 12.754705000000001],
     [80.070269, 12.754087999999998]]]},
  'PRODUCT_ID': 'S2X_MSIL2A'}}

In [14]:
SZRender = spatial.genSZRender(ndviLayerCol, ndvi, resource)

In [ ]:
# SZRender.geometry().getInfo()

In [15]:
SZRenderArea = SZRender.map(spatial.setArea)

In [24]:
SZRenderArea.getInfo()

{'type': 'FeatureCollection',
 'columns': {'area': 'Float', 'layerID': 'Number', 'meanNDVI': 'Float'},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Polygon',
    'coordinates': [[[80.0695182531196, 12.750884772154988],
      [80.0695182531196, 12.750857822696466],
      [80.06950926996674, 12.750857822696466],
      [80.06950926996674, 12.750821890085101],
      [80.0695002868139, 12.750821890085101],
      [80.0695002868139, 12.750785957473735],
      [80.06949130366107, 12.750785957473735],
      [80.06949130366107, 12.75075002486237],
      [80.06948232050823, 12.75075002486237],
      [80.06948232050823, 12.750714092251005],
      [80.06947333735539, 12.750714092251005],
      [80.06947333735539, 12.7506691764868],
      [80.06946435420254, 12.7506691764868],
      [80.06946435420254, 12.750633243875434],
      [80.0694553710497, 12.750633243875434],
      [80.0694553710497, 12.75059731126407],
      [80.06944638789686, 12.75059731126407],
     

In [25]:
exportList = {
    "RawData": rawData, 
    "NDVIRender": ndviLayerCol, 
    "NDMIRender": ndmiLayerCol, 
    "SZRender": SZRenderArea
}

In [26]:
tasklist = []
for exportName, exportObj in exportList.items():
    filePrefix = f"{remotefileName}-2-TestExportAcq"
    task = ee.batch.Export.table.toCloudStorage(collection= exportObj, 
                                                bucket="antpod-apgis-exports", 
                                                fileNamePrefix=f"{filePrefix}-{exportName}",
                                                fileFormat= 'GeoJSON')
    tasklist.append(task)

In [27]:
for task in tasklist:
    task.start()

In [30]:
tasklist[3].status()

{'state': 'COMPLETED',
 'description': 'myExportTableTask',
 'creation_timestamp_ms': 1599898312607,
 'update_timestamp_ms': 1599898320364,
 'start_timestamp_ms': 1599898318540,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://console.developers.google.com/storage/browser/antpod-apgis-exports/'],
 'id': 'GMZRRRWYDKFM3NNFNO3YRRUA',
 'name': 'projects/earthengine-legacy/operations/GMZRRRWYDKFM3NNFNO3YRRUA'}